![](task5.jpg)

## 1.	Обучите простую рекуррентную нейронную сеть (без GRU/LSTM, без внимания) решать задачу дешифровки шифра Цезаря.

<br>1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на N каждой буквы).
Например, если N=2, то буква A переходит в букву C.
Можно поиграться с языком на выбор (немецкий, русский и т. д.).
<br>2. Создать архитектуру рекуррентной нейронной сети.
<br>3. Обучить её (вход — зашифрованная фраза, выход — дешифрованная фраза).
<br>4. Проверить качество модели.

In [1]:
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, LSTM, GRU, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import random
from nltk.translate.bleu_score import sentence_bleu

In [4]:
import nltk
from nltk.corpus import gutenberg

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/anetta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
def caesar_cipher(text, shift, alphabet):
    """
    Функция для шифрования текста с использованием шифра Цезаря.
    
    :param text: Исходный текст для шифрования.
    :param shift: Смещение по алфавиту.
    :param alphabet: Алфавит, используемый для шифрования.
    :return: Зашифрованный текст.
    """
    shifted_text = "".join(
        alphabet[(alphabet.index(c) + shift) % len(alphabet)] if c in alphabet else c for c in text
    )
    return shifted_text

In [7]:
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [8]:
caesar_cipher('абвгде', 1, alphabet)

'бвгдеё'

In [9]:
nltk.download('gutenberg')
text_samples = gutenberg.sents('austen-emma.txt')[:1000]  
text_samples = [" ".join(sent).lower() for sent in text_samples if len(sent) > 5]

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/anetta/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [10]:
text_samples[:2]

['[ emma by jane austen 1816 ]',
 'emma woodhouse , handsome , clever , and rich , with a comfortable home and happy disposition , seemed to unite some of the best blessings of existence ; and had lived nearly twenty - one years in the world with very little to distress or vex her .']

In [11]:
alphabet = "abcdefghijklmnopqrstuvwxyz "
shift = 3

In [12]:
X = [caesar_cipher(text, shift, alphabet) for text in text_samples]
y = text_samples

In [13]:
def text_to_sequence(text, char_to_index):
    """
    Преобразует текст в последовательность индексов на основе заданного словаря символов.
    
    :param text: Входной текст.
    :param char_to_index: Словарь, сопоставляющий символы их индексам.
    :return: Список индексов, представляющих входной текст.
    """
    return [char_to_index[char] for char in text if char in char_to_index]

In [14]:
char_to_index = {char: i for i, char in enumerate(alphabet)}
index_to_char = {i: char for char, i in char_to_index.items()}

In [15]:
X_seq = [text_to_sequence(text, char_to_index) for text in X]
y_seq = [text_to_sequence(text, char_to_index) for text in y]

In [16]:
#Заполним последовательности до одинаковой длины
max_len = max(len(seq) for seq in X_seq)
X_seq = tf.keras.preprocessing.sequence.pad_sequences(X_seq, maxlen=max_len, padding='post')
y_seq = tf.keras.preprocessing.sequence.pad_sequences(y_seq, maxlen=max_len, padding='post')

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [18]:
vocab_size = len(alphabet)
embedding_dim = 8
hidden_units = 64

In [19]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    SimpleRNN(hidden_units, return_sequences=True),
    Dense(vocab_size, activation='softmax')
])

In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, epochs=5, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.8170 - loss: 1.0586 - val_accuracy: 0.8826 - val_loss: 0.4033
Epoch 2/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.9011 - loss: 0.3526 - val_accuracy: 0.9387 - val_loss: 0.2611
Epoch 3/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.9510 - loss: 0.2214 - val_accuracy: 0.9729 - val_loss: 0.1315
Epoch 4/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - accuracy: 0.9802 - loss: 0.1022 - val_accuracy: 0.9898 - val_loss: 0.0562
Epoch 5/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.9925 - loss: 0.0434 - val_accuracy: 0.9959 - val_loss: 0.0274


In [22]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.4f}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9960 - loss: 0.0275
Test accuracy: 0.9959


In [23]:
def decode_sequence(model, input_text, char_to_index, index_to_char, max_len):
    """
    Декодирует зашифрованную последовательность с помощью обученной модели.
    
    :param model: Обученная нейросеть.
    :param input_text: Зашифрованный текст.
    :param char_to_index: Словарь преобразования символов в индексы.
    :param index_to_char: Словарь преобразования индексов в символы.
    :param max_len: Максимальная длина последовательности.
    :return: Декодированный текст.
    """
    input_seq = text_to_sequence(input_text, char_to_index)
    input_seq = tf.keras.preprocessing.sequence.pad_sequences([input_seq], maxlen=max_len, padding='post')
    predicted_seq = model.predict(input_seq)
    predicted_chars = [index_to_char[np.argmax(vec)] for vec in predicted_seq[0]]
    return "".join(predicted_chars).strip()

In [24]:
text = "hello world"
example_encrypted = caesar_cipher(text, shift, alphabet)
example_decrypted = decode_sequence(model, example_encrypted, char_to_index, index_to_char, len(text))
print(f'Зашифрованное: {example_encrypted}')
print(f'Расшифрованное: {example_decrypted}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Зашифрованное: khoorczruog
Расшифрованное: hello world


## 2.	Сгенерировать последовательности, которые состоят из цифр (от 0 до 9) и задаются следующим образом:

 x  - последовательность цифр,

$$ y_1 = x_1 $$

$$ y_i = x_i + x_{i-1} $$

Если  $$ y_i \geq 10 $$ , то  $$ y_i = y_i - 10 $$
Научить модель рекуррентной нейронной сети предсказывать  $ y_i $  по  $ x_i $
Использовать: RNN, LSTM, GRU.

In [25]:
def generate_sequences(n_samples=10000, seq_length=10):
    """
    Генерирует случайные последовательности, которые состоят из цифр (от 0 до 9).
    
        n_samples (int): Количество последовательностей.
        seq_length (int): Длина каждой последовательности.
    
    Return:
        X (np.array): Исходные последовательности.
        Y (np.array): Преобразованные последовательности.
    """
    X, Y = [], []
    for _ in range(n_samples):
        x = np.random.randint(0, 10, seq_length)
        y = np.zeros_like(x)
        y[0] = x[0]
        for i in range(1, seq_length):
            y[i] = (x[i] + x[i - 1]) % 10  
        X.append(x)
        Y.append(y)
    return np.array(X), np.array(Y)

In [26]:
X, Y = generate_sequences()

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [28]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [29]:
X[:1]

array([[4, 3, 0, 3, 1, 6, 6, 0, 9, 3]])

In [30]:
Y[:1]

array([[4, 7, 3, 3, 4, 7, 2, 6, 9, 2]])

In [31]:
X.shape, Y.shape

((10000, 10), (10000, 10))

In [32]:
metrics = {}

In [33]:
from tensorflow.keras.layers import Bidirectional

#model_rnn.add(Bidirectional(SimpleRNN(50, activation='relu', return_sequences=True)))

In [34]:
#RNN
model_rnn = Sequential()
model_rnn.add(SimpleRNN(50, activation='relu', return_sequences=True))
model_rnn.add(Dense(10, activation='softmax'))
model_rnn.compile(optimizer='adam', metrics=['accuracy'], loss='sparse_categorical_crossentropy')

In [35]:
model_rnn.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1244 - loss: 2.3298 - val_accuracy: 0.1837 - val_loss: 2.1320
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2165 - loss: 2.0665 - val_accuracy: 0.3350 - val_loss: 1.8299
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3823 - loss: 1.7520 - val_accuracy: 0.5009 - val_loss: 1.5434
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5269 - loss: 1.4802 - val_accuracy: 0.5977 - val_loss: 1.3258
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6217 - loss: 1.2671 - val_accuracy: 0.6382 - val_loss: 1.1428
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6864 - loss: 1.0855 - val_accuracy: 0.7418 - val_loss: 0.9796
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7548 - loss: 0.9430 - val_accuracy: 0.7998 - val_loss: 0.8630
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8075 - loss: 0.8273 - val_accuracy: 0.

In [36]:
loss, accuracy = model_rnn.evaluate(X_test, Y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - accuracy: 0.8942 - loss: 0.5969


In [37]:
metrics['RNN'] = accuracy

In [38]:
#LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', return_sequences=True))
model_lstm.add(Dense(10, activation='softmax'))
model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [39]:
model_lstm.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1024 - loss: 2.3094 - val_accuracy: 0.1375 - val_loss: 2.2435
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1606 - loss: 2.1786 - val_accuracy: 0.2971 - val_loss: 1.8471
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3652 - loss: 1.6742 - val_accuracy: 0.5511 - val_loss: 1.2558
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6171 - loss: 1.1418 - val_accuracy: 0.7281 - val_loss: 0.8943
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7618 - loss: 0.8341 - val_accuracy: 0.8315 - val_loss: 0.6863
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8407 - loss: 0.6446 - val_accuracy: 0.8480 - val_loss: 0.5654
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8922 - loss: 0.5017 - val_accuracy: 0.9077 - val_loss: 0.4557
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9326 - loss: 0.4044 - val_accuracy: 0.

In [40]:
loss, accuracy = model_lstm.evaluate(X_test, Y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - accuracy: 0.9571 - loss: 0.2709


In [41]:
metrics['LSTM'] = accuracy

In [42]:
#GRU
model_gru = Sequential()
model_gru.add(GRU(50, activation='relu', return_sequences=True))
model_gru.add(Dense(10, activation='softmax'))
model_gru.compile(optimizer='adam', metrics=['accuracy'], loss='sparse_categorical_crossentropy')

In [43]:
model_gru.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1237 - loss: 2.2951 - val_accuracy: 0.2224 - val_loss: 2.0837
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2831 - loss: 1.9352 - val_accuracy: 0.4470 - val_loss: 1.4668
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5483 - loss: 1.3152 - val_accuracy: 0.7073 - val_loss: 0.9713
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7413 - loss: 0.8856 - val_accuracy: 0.7981 - val_loss: 0.7002
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8318 - loss: 0.6373 - val_accuracy: 0.8748 - val_loss: 0.5314
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8877 - loss: 0.4846 - val_accuracy: 0.9089 - val_loss: 0.4167
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9222 - loss: 0.3826 - val_accuracy: 0.9289 - val_loss: 0.3402
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9456 - loss: 0.3114 - val_accuracy: 0.

In [44]:
loss, accuracy = model_gru.evaluate(X_test, Y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.9527 - loss: 0.2646


In [45]:
metrics['GRU'] = accuracy

In [46]:
metrics

{'RNN': 0.8922499418258667,
 'LSTM': 0.9569500684738159,
 'GRU': 0.9509499669075012}

Модели показали отличные результаты – они справляются с запоминанием последовательностей и корректно предсказывают $ y_i $.

## Решить задачу машинного перевода, выбрав свой язык:
	•	Формируем датасет с исходного языка на целевой (код прописать в классе).
	•	Строим архитектуру нейронной сети.
	•	Обучаем.
	•	Проверить качество с помощью метрики BLEU.

In [47]:
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings
import time
import math

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
#Токен начала предложения
SOS_token = 0
#Токен конца предложения
EOS_token = 1

class LanguageVocabulary(object):
    """
    Класс для хранения информации о языке и управления словарем токенов.

    Атрибуты:
        name (str): Название языка.
        word2index (dict): Словарь, отображающий слово в его числовой индекс.
        word2count (dict): Словарь, содержащий количество встречаемости слов.
        index2word (dict): Обратный словарь для word2index.
        n_words (int): Количество уникальных слов (включая специальные токены SOS и EOS).
    """
    def __init__(self, name):
        """
        Инициализация объекта словаря языка.
        
        Аргументы:
            name (str): Название языка.
        """
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2

    def add_sentence(self, sentence):
        """
        Добавляет слова из предложения в словарь.
        
        Аргументы:
            sentence (str): Входное предложение.
        """
        for word in sentence.split(' '):
            self.add_word(word)


    def add_word(self, word):
        """
        Добавляет слово в словарь, если оно отсутствует, 
        либо увеличивает его счетчик.
        
        Аргументы:
            word (str): Слово для добавления.
        """
        if word not in self.word2index:
            
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [49]:
def unicode_to_ascii(s):
    """
    Преобразует строку в ASCII, удаляя диакритические знаки.

    Аргументы:
        s (str): Входная строка в юникоде.

    Возвращает:
        str: Строка в ASCII.
    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [50]:
def normalize_string(s):
    """
    Нормализует строку:
    1. Преобразует в ASCII.
    2. Приводит к нижнему регистру и убирает пробельные символы с краев.
    3. Отделяет знаки препинания (!, ?, .) пробелами.
    4. Удаляет все символы, кроме латинских букв и знаков препинания.

    Аргументы:
        s (str): Входная строка.

    Возвращает:
        str: Нормализованная строка.
    """
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [51]:
def read_languages(lang1, lang2, reverse=False):
    """
    Считывает данные из файла корпуса, разбивает их на пары предложений 
    и нормализует текст.
    
    Аргументы:
        lang1 (str): Код первого языка 
        lang2 (str): Код второго языка 
        reverse (bool, optional): Если True, меняет порядок языков (по умолчанию False).
    
    Возвращает:
        tuple: Кортеж из трех элементов:
            - input_lang (LanguageVocabulary): Объект словаря для исходного языка.
            - output_lang (LanguageVocabulary): Объект словаря для целевого языка.
            - pairs (list): Список пар предложений (исходное, целевое).
    """
    print("Reading lines...")
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = LanguageVocabulary(lang2)
        output_lang = LanguageVocabulary(lang1)
    else:
        input_lang = LanguageVocabulary(lang1)
        output_lang = LanguageVocabulary(lang2)
    return input_lang, output_lang, pairs

In [52]:
MAX_LENGTH = 10
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

In [53]:
def filter_pair(p):
    """
    Фильтрует пары предложений, оставляя только те, которые соответствуют заданным условиям.
    
    Условия:
        - Длина предложений (в словах) должна быть меньше MAX_LENGTH.
        - Целевое предложение должно начинаться с одного из предопределенных префиксов eng_prefixes.
    
    Аргументы:
        p (tuple): Кортеж из двух строк (исходное предложение, целевое предложение).
    
    Возвращает:
        bool: True, если пара проходит фильтрацию, иначе False.
    """
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

In [54]:
def filter_pairs(pairs):
    """
    Применяет фильтр к списку пар предложений, оставляя только те, 
    которые соответствуют заданным условиям.
    
    Аргументы:
        pairs (list): Список пар предложений (кортежи строк).
    
    Возвращает:
        list: Отфильтрованный список пар предложений.
    """
    return [pair for pair in pairs if filter_pair(pair)]

In [55]:
def prepare_data(lang1, lang2, reverse=False):
    """
    Загружает, фильтрует и подготавливает данные для обучения модели перевода.
    
    Аргументы:
        lang1 (str): Код первого языка 
        lang2 (str): Код второго языка 
        reverse (bool, optional): Если True, меняет порядок языков (по умолчанию False).
    
    Возвращает:
        tuple: Кортеж из трех элементов:
            - input_lang (LanguageVocabulary): Объект словаря для исходного языка.
            - output_lang (LanguageVocabulary): Объект словаря для целевого языка.
            - pairs (list): Отфильтрованный список пар предложений (исходное, целевое).
    """
    input_lang, output_lang, pairs = read_languages(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filter_pairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [56]:
input_lang, output_lang, pairs = prepare_data('eng', 'fra', True)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10853 sentence pairs
Counting words...
Counted words:
fra 4489
eng 2925


In [57]:
class EncoderRNN(nn.Module):
    """
    Кодировщик RNN на основе GRU для последовательного машинного перевода.
    
    Атрибуты:
        hidden_size (int): Размер скрытого состояния GRU.
        embedding (nn.Embedding): Слой эмбеддингов для преобразования входных токенов в векторное представление.
        gru (nn.GRU): Глубокая рекуррентная сеть GRU, принимающая эмбеддинги и скрытые состояния.
    """
    def __init__(self, input_size, hidden_size):
        """
        Инициализирует кодировщик.
        
        Аргументы:
            input_size (int): Размер входного словаря.
            hidden_size (int): Размер скрытого состояния GRU.
        """
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        """
        Пропускает входное слово через эмбеддинги и GRU.
        
        Аргументы:
            input (torch.Tensor): Входной тензор токенов (размерность: 1).
            hidden (torch.Tensor): Скрытое состояние (размерность: 1, 1, hidden_size).
        
        Возвращает:
            tuple: Выход из GRU и обновленное скрытое состояние.
        """
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        """
        Инициализирует скрытое состояние нулями.
        
        Возвращает:
            torch.Tensor: Тензор нулей размерности (1, 1, hidden_size).
        """
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [58]:
class DecoderRNN(nn.Module):
    """
    Декодер RNN на основе GRU для последовательного машинного перевода.
    
    Атрибуты:
        hidden_size (int): Размер скрытого состояния GRU.
        embedding (nn.Embedding): Слой эмбеддингов для представления токенов в виде векторов.
        gru (nn.GRU): Глубокая рекуррентная сеть GRU, принимающая эмбеддинги и скрытые состояния.
        out (nn.Linear): Полносвязный слой для преобразования выходных данных в размерность словаря.
        softmax (nn.LogSoftmax): Функция активации для получения распределения вероятностей по выходным токенам.
    """
    
    def __init__(self, hidden_size, output_size):
        """
        Инициализирует декодер.
        
        Аргументы:
            hidden_size (int): Размер скрытого состояния GRU.
            output_size (int): Размер выходного словаря.
        """
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input, hidden):
        """
        Пропускает входное слово через эмбеддинги, GRU и 
        полносвязный слой для получения предсказанного слова.
        
        Аргументы:
            input (torch.Tensor): Входной тензор токенов (размерность: 1).
            hidden (torch.Tensor): Скрытое состояние (размерность: 1, 1, hidden_size).
        
        Возвращает:
            tuple: Выход из GRU и обновленное скрытое состояние.
        """
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        """
        Инициализирует скрытое состояние нулями.
        
        Возвращает:
            torch.Tensor: Тензор нулей размерности (1, 1, hidden_size).
        """
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [59]:
def indexesFromSentence(lang, sentence):
    """
    Преобразует предложение в список индексов токенов на основе словаря языка.
    
    Аргументы:
        lang (LanguageVocabulary): Объект словаря языка, содержащий маппинг слов в индексы.
        sentence (str): Входное предложение, разбитое на слова.
    
    Возвращает:
        list: Список индексов слов, соответствующих токенам в предложении.
    """
    return [lang.word2index[word] for word in sentence.split(' ')]

In [60]:
def tensorFromSentence(lang, sentence):
    """
    Преобразует предложение в тензор PyTorch, добавляя токен конца предложения (EOS).
    
    Аргументы:
        lang (LanguageVocabulary): Объект словаря языка, содержащий маппинг слов в индексы.
        sentence (str): Входное предложение, разбитое на слова.
    
    Возвращает:
        torch.Tensor: Тензор с индексами слов (размерность: [кол-во слов, 1]).
    """
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [61]:
def tensorsFromPair(pair):
    """
    Преобразует пару предложений (входное и целевое) в тензоры.
    
    Аргументы:
        pair (tuple): Кортеж из двух строк (входное предложение, целевое предложение).
    
    Возвращает:
        tuple: Кортеж из двух тензоров:
            - input_tensor (torch.Tensor): Тензор с индексами слов для входного языка.
            - target_tensor (torch.Tensor): Тензор с индексами слов для целевого языка.
    """
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [62]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    """
    Выполняет обучение модели машинного перевода с использованием механизма Teacher Forcing.
    
    Аргументы:
        input_tensor (torch.Tensor): Входное предложение в виде тензора индексов.
        target_tensor (torch.Tensor): Целевое предложение в виде тензора индексов.
        encoder (nn.Module): Экземпляр энкодера (EncoderRNN).
        decoder (nn.Module): Экземпляр декодера (DecoderRNN).
        encoder_optimizer (torch.optim.Optimizer): Оптимизатор для энкодера.
        decoder_optimizer (torch.optim.Optimizer): Оптимизатор для декодера.
        criterion (nn.Module): Функция потерь (например, NLLLoss).
        max_length (int, optional): Максимальная длина последовательности (по умолчанию MAX_LENGTH).
    
    Возвращает:
        float: Потери (loss) на текущем примере обучения.
    """
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length 

In [63]:
def asMinutes(s):
    """
    Преобразует количество секунд в строку формата 'Xm Ys'.
    
    Аргументы:
        s (float): Время в секундах.
    
    Возвращает:
        str: Строка, представляющая время в минутах и секундах.
    """
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [64]:
def timeSince(since, percent):
    """
    Вычисляет прошедшее время и оценивает оставшееся время на основе процента выполнения.
    
    Аргументы:
        since (float): Временная метка начала (time.time()).
        percent (float): Процент выполнения (от 0 до 1).
    
    Возвращает:
        str: Форматированная строка с прошедшим временем и оставшимся временем.
    """
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- eta: %s)' % (asMinutes(s), asMinutes(rs))

In [65]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01):
    """
    Обучает модель машинного перевода на заданном количестве итераций.
    
    Аргументы:
        encoder (nn.Module): Экземпляр энкодера (EncoderRNN).
        decoder (nn.Module): Экземпляр декодера (DecoderRNN).
        n_iters (int): Количество итераций обучения.
        print_every (int, optional): Как часто выводить среднее значение потерь (по умолчанию 1000).
        learning_rate (float, optional): Скорость обучения (по умолчанию 0.01).
    """
    start = time.time()
    
    print_loss_total = 0 
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for epoch in range(1, n_iters + 1):
        training_pair = training_pairs[epoch - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_iters),
                                         epoch, epoch / n_iters * 100, print_loss_avg))


In [66]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    """
    Выполняет машинный перевод входного предложения с использованием обученного энкодера и декодера.
    
    Аргументы:
        encoder (nn.Module): Экземпляр энкодера (EncoderRNN).
        decoder (nn.Module): Экземпляр декодера (DecoderRNN).
        sentence (str): Входное предложение для перевода.
        max_length (int, optional): Максимальная длина выходного предложения (по умолчанию MAX_LENGTH).
    
    Возвращает:
        list: Список предсказанных слов (включая EOS при завершении перевода).
    """
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for i in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
            encoder_outputs[i] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = [] 

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

In [67]:
def evaluateRandomly(encoder, decoder, n=10):
    """
    Выбирает случайные пары предложений и оценивает качество перевода с помощью модели.

    Аргументы:
        encoder (nn.Module): Экземпляр энкодера (EncoderRNN).
        decoder (nn.Module): Экземпляр декодера (DecoderRNN).
        n (int, optional): Количество примеров для оценки (по умолчанию 10).

    Возвращает:
        None
    """
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [68]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 24s (- eta: 19m 39s) (5000 6%) 2.9713
2m 49s (- eta: 18m 24s) (10000 13%) 2.4002
4m 15s (- eta: 17m 0s) (15000 20%) 2.0644
5m 41s (- eta: 15m 38s) (20000 26%) 1.8334
7m 9s (- eta: 14m 18s) (25000 33%) 1.6080
8m 35s (- eta: 12m 53s) (30000 40%) 1.4543
10m 1s (- eta: 11m 27s) (35000 46%) 1.2807
11m 27s (- eta: 10m 1s) (40000 53%) 1.1381
12m 55s (- eta: 8m 36s) (45000 60%) 1.0428
14m 21s (- eta: 7m 10s) (50000 66%) 0.9475
15m 48s (- eta: 5m 45s) (55000 73%) 0.8424
17m 16s (- eta: 4m 19s) (60000 80%) 0.7278
18m 43s (- eta: 2m 52s) (65000 86%) 0.6734
20m 16s (- eta: 1m 26s) (70000 93%) 0.6180
21m 44s (- eta: 0m 0s) (75000 100%) 0.5467


In [70]:
evaluateRandomly(encoder1, decoder1)

> vous m empoisonnez .
= you re poisoning me .
< you re avoiding me . <EOS>

> tu es tres serviable .
= you re very helpful .
< you re very helpful . <EOS>

> tu es trop vieille pour moi .
= you re too old for me .
< you re too old for me . <EOS>

> je suis desolee mais vous devez partir .
= i m sorry but you need to leave .
< i m sorry but you need to leave . <EOS>

> nous sommes serieux .
= we re serious .
< we re serious . <EOS>

> vous n etes pas vous meme aujourd hui .
= you re not yourself today .
< you re not yourself today . <EOS>

> je te donne ce que tu veux .
= i m giving you what you want .
< i m just you didn t . . <EOS>

> nous ne sommes pas jeunes .
= we re not young .
< we re not young . <EOS>

> nous sommes tous retraites .
= we re all retired .
< we re all crazy . <EOS>

> elle est d un naturel calme .
= she s a quiet person .
< she s a quiet person . <EOS>

